In [1]:
import numpy as np

import keras

from keras.models import load_model
from keras_applications import inception_v3
from keras.preprocessing import image

decode_predictions = inception_v3.decode_predictions
preprocess_input = inception_v3.preprocess_input

Using TensorFlow backend.


In [2]:
model = keras.applications.inception_v3.InceptionV3(include_top=True, 
                                                    weights='imagenet', 
                                                    input_tensor=None, 
                                                    input_shape=(299, 299, 3), 
                                                    pooling=None, 
                                                    classes=1000)

In [16]:
model.save('InceptionV3.h5')

In [10]:
model = load_model('./InceptionV3/InceptionV3.h5')

/Users/runshengsong/anaconda3/envs/insight/lib/python2.7/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
img_path = './sharks2.png'
img = image.load_img(img_path, target_size=(299, 299))

In [6]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [7]:
x.shape

(1, 299, 299, 3)

In [60]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n01491361', u'tiger_shark', 0.72572047), (u'n10565667', u'scuba_diver', 0.025161441), (u'n01484850', u'great_white_shark', 0.01840023), (u'n01494475', u'hammerhead', 0.017019149), (u'n01496331', u'electric_ray', 0.002886214)]])


In [12]:
print keras.__version__

2.2.0
